# Initialisation

In [ ]:
from __future__ import annotations

import locale
import random
from pathlib import Path

import mqt.qubomaker as qm
from mqt.qubomaker import Graph
from mqt.qubomaker import pathfinder as pf

In [ ]:
with Path("input/graph").open(encoding=locale.getpreferredencoding(False)) as file:
    graph = Graph.read(file)
graph.plot()

# QUBO Construction

### Parameters

We begin by setting up our parameters for the QUBOGenerator. The `PathFindingQUBOGenerator` supports the following parameters:

- `encoding_type`: One of `ONE_HOT`, `DOMAIN_WALL`, or `BINARY`. the encoding for the binary variables in  the QUBO formulation.
- `n_paths`: The number of paths to be searched.
- `max_path_length`: The maximum length of a path to be searched.
- `loops`: Indicates, whether the searched path should be a loop.

In [ ]:
encoding_type = pf.EncodingType.DOMAIN_WALL
n_paths = 1
max_path_length = graph.n_vertices
loops = True

settings = pf.PathFindingQUBOGeneratorSettings(encoding_type, n_paths, max_path_length, loops)

Using these settings, we can now create a new `PathFindingQUBOGenerator`:

In [ ]:
generator = pf.PathFindingQUBOGenerator(
    objective_function=pf.MinimizePathLength(path_ids=[1]),
    graph=graph,
    settings=settings,
)

### Constraints

We can add constraints to the QUBO generator with the `add_constraint` method. For this example, we add two constraints:

1) `PathIsValid`: Enforces that the found path is actually valid (i.e. all edges in it exist).
2) `PathContainsVerticesExactlyOnce`: Enforces that the given vertices appear exactly once in the path.

In [ ]:
generator.add_constraint(pf.PathIsValid(path_ids=[1]))
generator.add_constraint(pf.PathContainsVerticesExactlyOnce(vertex_ids=graph.all_vertices, path_ids=[1]))

### Generate QUBO Formulation

There are several ways to generate the problem's QUBO formulation.

`QUBOGenerator.construct()` generates a simplified mathematical expression for the problem: 

In [ ]:
generator.construct()

`QUBOGenerator.construct_expansion()` generates an expanded formula of the form

$C_{1,1}x_1 + C_{1,2}x_1x_2 + C_{1,3}x_1x_3 + ... + C_{n-1,n}x_{n-1}x_n$

In [ ]:
generator.construct_expansion()

`QUBOGenerator.construct_qubo_matrix()` generates the QUBO formulation as a QUBO matrix $Q$ such that the QUBO problem can be formulated as

$$\mathbf{x}^* = \argmin_\mathbf{x} \mathbf{x}^T Q \mathbf{x}$$

In [ ]:
A = generator.construct_qubo_matrix()
qm.print_matrix(A)

# Test Results

### Brute Force Optimization

We offer a naive brute-force optimization method to test simple QUBO formulations. It will generate the optimal assignment vector $\mathbf{x}^*$ which can be passed to the method `QUBOGenerator.decode_bit_array(...)` to translate it into a readable solution of the problem (in this case a list of paths).

In [ ]:
import numpy as np

(best_test, best_score) = qm.optimize_classically(A)

x = np.array(best_test)
pth = generator.decode_bit_array(best_test)
print(pth)

### Operator: Classical Eigensolver

The method `QUBOGenerator.construct_operator()` generates the QUBO formulation as a quantum operator that can be used for optimuzation. Using qiskit, we can compute its minimal eigenvalue using classical methods or quantum algorithms.

In [ ]:
from typing import TYPE_CHECKING

import numpy as np
import numpy.typing as npt
from qiskit.result import QuasiDistribution
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver

if TYPE_CHECKING:
    from qiskit.quantum_info import Statevector


def bitfield(n: int, width: int) -> list[int]:
    result = np.binary_repr(n, width)
    return [int(digit) for digit in result]


def sample_most_likely(
    state_vector: QuasiDistribution | Statevector | dict[str, float],
) -> npt.NDArray[np.int_ | np.float64]:
    """Compute the most likely binary string from state vector.
    Args:
        state_vector: State vector or quasi-distribution.

    Returns:
        Binary string as an array of ints.
    """
    values = (
        list(state_vector.values())
        if isinstance(state_vector, QuasiDistribution)
        else [state_vector[key] for key in state_vector]
        if isinstance(state_vector, dict)
        else state_vector
    )
    n = int(np.log2(len(values)))
    k = np.argmax(np.abs(values))
    x = bitfield(k, n)
    x.reverse()
    return np.asarray(x)


op = generator.construct_operator()

npme = NumPyMinimumEigensolver()
result = npme.compute_minimum_eigenvalue(op)
x = sample_most_likely(result.eigenstate)
print(x)
print(generator.decode_bit_array(x))
print(result.eigenvalue)

## Quantum Circuits

The constructed operator can be used to create quantum circuits that solve the optimization problem.

### QAOA

In [ ]:
seed = random.randint(10000, 20000)
(qaoa, op) = generator.construct_qaoa(seed=seed)

result = qaoa.compute_minimum_eigenvalue(op)
x = sample_most_likely(result.eigenstate)
print(generator.decode_bit_array(x))
print(result.eigenvalue)

# Other Features

_Use the JSON input format_

In [ ]:
with Path.open("input/tsp.json") as file:
    generator_new = pf.PathFindingQUBOGenerator.from_json(file.read(), graph)

A = generator_new.construct_qubo_matrix()
(best_test, best_score) = qm.optimize_classically(A)

pth = generator_new.decode_bit_array(best_test)
print(pth)

_Use Encoding suggestion_

The `PathFindingQUBOGenerator` supports the suggestion of the optimal encoding for a given problem instance, based on the number of required binary variables. 

In [ ]:
with Path.open("input/tsp.json") as file:
    print(pf.PathFindingQUBOGenerator.suggest_encoding(file.read(), graph))

## Different constraints

_Also define the starting vertex of the path_

In [ ]:
generator_new = pf.PathFindingQUBOGenerator(pf.MinimizePathLength([1]), graph, settings)
generator_new.add_constraint(pf.PathIsValid([1]))
generator_new.add_constraint(pf.PathContainsVerticesExactlyOnce(graph.all_vertices, [1]))


generator_new.add_constraint(pf.PathStartsAt([3], 1))


A = generator_new.construct_qubo_matrix()
(best_test, best_score) = qm.optimize_classically(A)

pth = generator_new.decode_bit_array(best_test)
print(pth)

_Find the shortest paths $\pi_1$ and $\pi_2$ from $s_p$ to $t_p$ respectively that don't intersect_

In [ ]:
(s1, t1) = 1, 5
(s2, t2) = 2, 6

settings = pf.PathFindingQUBOGeneratorSettings(encoding_type, n_paths=2, max_path_length=max_path_length, loops=True)
generator_new = pf.PathFindingQUBOGenerator(pf.MinimizePathLength([1, 2]), graph, settings)
generator_new.add_constraint(pf.PathIsValid([1, 2]))
generator_new.add_constraint(pf.PathStartsAt([s1], 1))
generator_new.add_constraint(pf.PathStartsAt([s2], 2))
generator_new.add_constraint(pf.PathEndsAt([t1], 1))
generator_new.add_constraint(pf.PathEndsAt([t2], 2))
generator_new.add_constraint(pf.PathsShareNoVertices(1, 2))